### Interacive Visualization (video games) with arcade

<img src="https://api.arcade.academy/en/stable/_images/template_platformer1.png" width=600>

[example video here](https://vimeo.com/662273899)

(c) 2023 Timothy Becker, Department of Computer Science <br><br>
<img src="https://www.conncoll.edu/media/website-media/visualidentity/images/1Line-LogoSig-Color.jpg" width="200">

### Preliminary
For the upcoming programming assignment (Program_5.ipynb) you will be working with the arcade python video game library that facilitates several common video game workflows and functions: Tiled Maps, Hit detection, physics, sound, etc...

### (I) Install arcade
#### Install the library using either conda or pip
The commands below should allow you to install the arcade library. This library is object orientated and all of your efforts spent in CH10 and CH12 will pay off in that using this library will make some sense. 

In [1]:
#%conda install arcade
#%pip install arcade #alterate way to install

### (II) Run the game (now has controls/gravity)
First we will run the game and then make some modifications to get some animation to happen for the main character. This starts by using the femaleAdventurer_idle.png image, but we will get animation using the images that say walk.

In [2]:
from PIL import Image
image = Image.open('angrybirdsss.png')
new_image = image.resize((90,90))
new_image.save('Angrybirds.png')

image = Image.open('catapult.png')
new_image = image.resize((150,150))
new_image.save('C2atapult.png')



In [3]:
import arcade
import glob


class PlayerCharacter(arcade.Sprite): #player class has sprite, hitbox, etc
    def __init__(self):
        super().__init__()
        
        # Used for flipping between image sequences
        self.cur_texture = 0
        self.walk_textures = []
        self.scale = 1
        
        # Images from Kenney.nl's Asset Pack 3
        
        main_path = "Angrybirds*.png"
        for filename in sorted(glob.glob(main_path)):
            self.walk_textures += [arcade.load_texture(filename)]

        # Set the initial texture
        self.texture = self.walk_textures[0]

        # Hit box will be set based on the first image used. If you want to specify
        # a different hit box, you can do it like the code below.
        # set_hit_box = [[-22, -64], [22, -64], [22, 28], [-22, 28]]
        self.hit_box = self.texture.hit_box_points

    def update_animation(self, delta_time: float = 1 / 60):
        # Walking animation, code to sense if the PlayerCharacter is moving
        if self.change_x > 0 or self.change_x < 0: 
            print('\rx=%s,y=%s'%(int(self.center_x),int(self.center_y)))
        self.texture = self.walk_textures[self.cur_texture]

In [4]:
#use this area if you need to check out parts of the PlayerCharacter class
p1 = PlayerCharacter()
p1.color

(255, 255, 255)

In [5]:
# Constantscoin1.wav
import arcade
SCREEN_WIDTH = 1000
SCREEN_HEIGHT = 650
SCREEN_TITLE = "Platformer"

# Constants used to scale our sprites from original size = 1
TILE_SCALING = 0.5
COIN_SCALING = 0.5

# Movement speed of player, in pixels per frame
PLAYER_MOVEMENT_SPEED = 10
GRAVITY = 1
PLAYER_JUMP_SPEED = 20

class GameView(arcade.View): #load sounds from disk to RAM in constructor
    def __init__(self):
        super().__init__() #window
        
        self.tile_map, self.scene, self.player_sprite = None, None, None
        self.physics_engine, self.camera, self.gui_camera = None, None, None
        self.score = 0
        # Load sounds
        self.collect_coin_sound = arcade.load_sound("coin1.wav")
        self.jump_sound = arcade.load_sound("bird_sound.wav")
        arcade.set_background_color(arcade.csscolor.CORNFLOWER_BLUE)

        
   # def startscreen():
      #  self.v_box = arcade.gui.UIBoxLayout()
        #start_button = arcade.gui.UIFlatButton(text="Start Game", width=200)
        #self.v_box.add(start_button.with_space_around(bottom=20))
       # start_button.on_click = self.setup
        
    def setup(self): #starts the game (or restarts)
        # Keep track of the score
        self.score = 0
        
        # Set up the Cameras used to scroll the platforms, etc and Scene
       # self.camera = arcade.Camera(self.width, self.height)
        #self.gui_camera = arcade.Camera(self.width, self.height)
        self.scene = arcade.Scene() # Scene has everything you need in one structure
        
        self.player_sprite = PlayerCharacter()
        self.player_sprite.center_x = 128
        self.player_sprite.center_y = 228
        self.scene.add_sprite("Player", self.player_sprite)
        
        # This shows using a loop to place multiple ground sprites horizontally
        for x in range(0, 1250, 64):
            wall = arcade.Sprite("grassMid.png", TILE_SCALING)
            wall.center_x = x
            wall.center_y = 32
            self.scene.add_sprite("Platforms", wall)
            
        # Put some crates on the ground [x_pos,y_pos]
        coordinate_list = [[512, 96], [256, 96], [768, 96],[128, 96]] #these are [x,y coordinates]
        for coordinate in coordinate_list:
            # Add a crate on the ground
            wall = arcade.Sprite( "boxCrate_double.png", TILE_SCALING)
            wall.position = coordinate           #set the wall position
            self.scene.add_sprite("Platforms", wall) #add it to the scene

        # Add some coins to pick up
        for x in range(128, 1250, 256):
            coin = arcade.Sprite("pig2.png", COIN_SCALING)
            coin.center_x = x
            coin.center_y = 110
            self.scene.add_sprite("Coins", coin)

        # Create the 'physics engine'
        self.physics_engine = arcade.PhysicsEnginePlatformer(
            self.player_sprite,
            gravity_constant=GRAVITY, 
            walls=self.scene["Platforms"]
        )

    def on_draw(self):
        """Render the screen."""
        # Clear the screen to the background color
        self.clear()
        # Activate the game camera
       # self.camera.use()
        # Draw our Scene
        self.scene.draw()
        # Activate the GUI camera before drawing GUI elements
       # self.gui_camera.use()

        # Draw our score on the screen, scrolling it with the viewport
        score_text = f"Score: {self.score}"
        arcade.draw_text(score_text,10,10,arcade.csscolor.WHITE,18)
        
        catapult = arcade.Sprite("C2atapult.png")
        catapult.center_x = 128
        catapult.center_y = 130
        self.scene.add_sprite("Catapult", catapult)

    def on_key_press(self, key, modifiers):
        """Called whenever a key is pressed."""
        if key == arcade.key.UP:
            if self.physics_engine.can_jump():
                self.player_sprite.change_y = PLAYER_JUMP_SPEED
                self.player_sprite.change_x = PLAYER_MOVEMENT_SPEED
                arcade.play_sound(self.jump_sound)
        elif key == arcade.key.LEFT:
            self.player_sprite.change_x = -PLAYER_MOVEMENT_SPEED
        elif key == arcade.key.RIGHT:
            self.player_sprite.change_x = PLAYER_MOVEMENT_SPEED

    def on_key_release(self, key, modifiers):
        """Called when the user releases a key."""
        if key == arcade.key.LEFT:
            self.player_sprite.change_x = 0
        elif key == arcade.key.RIGHT:
            self.player_sprite.change_x = 0

    #def center_camera_to_player(self):
     #   screen_center_x = self.player_sprite.center_x - (self.camera.viewport_width / 2)
      #  screen_center_y = self.player_sprite.center_y - (self.camera.viewport_height / 2)
      #  if screen_center_x < 0: screen_center_x = 0
       # if screen_center_y < 0: screen_center_y = 0
       # player_centered = screen_center_x, screen_center_y
       # self.camera.move_to(player_centered)

    def on_update(self, delta_time):
        """Movement and game logic"""
        # Move the player with the physics engine
        self.physics_engine.update()
        
        # Update Animations
        self.scene.update_animation(delta_time, ["Player"])
        
        # See if we hit any coins
        coin_hit_list = arcade.check_for_collision_with_list(
            self.player_sprite, self.scene["Coins"]
        )

        # Loop through each coin we hit (if any) and remove it
        for coin in coin_hit_list:
            # Remove the coin
            coin.remove_from_sprite_lists()
            # Play a sound
            arcade.play_sound(self.collect_coin_sound)
            # Add one to the score
            self.score += 1

        # Position the camera
        #self.center_camera_to_player()

In [6]:
import time

class InstructionView(arcade.View):
    
    def on_show_view(self):
        """ This is run once when we switch to this view """
        arcade.set_background_color(arcade.color.DARK_BLUE_GRAY)
        self.background = arcade.load_texture("AGB3.jpeg")
        #self.song_sound = arcade.load_sound()
        self.music = arcade.load_sound('AB1.wav')
        self.media_player = self.music.play()     
        # Reset the viewport, necessary if we have a scrolling game and we need
        # to reset the viewport back to the start so we can see what we draw.
        arcade.set_viewport(0, self.window.width, 0, self.window.height)
       # arcade.play_sound(self.song_sound)
        
    def on_draw(self):
        """ Draw this view """
        self.clear()
        arcade.draw_lrwh_rectangle_textured(0, 0,
                                            SCREEN_WIDTH, SCREEN_HEIGHT,
                                            self.background)
        arcade.draw_text("Angry Birds Prototype", self.window.width / 2, self.window.height-80,
                         arcade.color.BLACK, font_size=80, anchor_x="center", font_name="Kenney Pixel")
        arcade.draw_text("Click to advance", self.window.width / 2, self.window.height-120,
                         arcade.color.BLACK, font_size=30, anchor_x="center",font_name="Kenney Pixel")
        
        
        
    def on_mouse_press(self, _x, _y, _button, _modifiers):
        """ If the user presses the mouse button, start the game. """
        self.media_player.pause()
        game_view = GameView()
        game_view.setup()
        self.window.show_view(game_view)
        
        

In [7]:
def main():
    
    window = arcade.Window(SCREEN_WIDTH, SCREEN_HEIGHT, SCREEN_TITLE)
    start_view = InstructionView()
    window.show_view(start_view)
    arcade.run()

In [ ]:
main()

2023-12-07 22:42:22.305 python[50611:1880675] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7fa80b881650>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2023-12-07 22:42:22.306 python[50611:1880675] Warning: Expected min height of view: (<NSButton: 0x7fa80bedde30>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2023-12-07 22:42:22.308 python[50611:1880675] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7fa80bedd830>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2023-12-07 22:42:22.309 python[50611:1880675] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7fa80bef2940>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.


x=138,y=191
x=148,y=209
x=158,y=226
x=168,y=242
x=178,y=257
x=188,y=271
x=198,y=284
x=208,y=296
x=218,y=307
x=228,y=317
x=238,y=326
x=248,y=334
x=258,y=341
x=268,y=347
x=278,y=352
x=288,y=356
x=298,y=359
x=308,y=361
x=318,y=362
x=328,y=362
x=338,y=361
x=348,y=359
x=358,y=356
x=368,y=352
x=378,y=347
x=388,y=341
x=398,y=334
x=408,y=326
x=418,y=317
x=428,y=307
x=438,y=296
x=448,y=284
x=458,y=271
x=468,y=257
x=478,y=242
x=488,y=226
x=498,y=209
x=508,y=191
x=518,y=172
x=528,y=172
x=538,y=172
x=548,y=172
x=558,y=172
x=568,y=171
x=578,y=169
x=588,y=166
x=598,y=162
x=608,y=157
x=618,y=151
x=628,y=144
x=638,y=136
x=648,y=127
x=658,y=117
x=668,y=108
x=678,y=108
x=688,y=108
x=691,y=108
x=691,y=108
x=691,y=108
x=691,y=108
x=691,y=108
x=691,y=108
x=691,y=108
x=691,y=108
x=691,y=108
x=691,y=108
x=691,y=108
x=691,y=108
x=691,y=108
x=691,y=108
x=691,y=108
x=691,y=108
x=691,y=108
x=691,y=108
x=691,y=108
x=691,y=108
x=691,y=108
x=691,y=108
x=691,y=108
x=691,y=108
x=691,y=108
x=691,y=108
x=691,y=108
x=69

### (III) Try to animate the sprites
Load the femaleAdventurer_walkx.png series of pictures by using glob into the walk_textures list. Then modify the update_animation method to change that sprite when the player hits a key (which will trigger change_x when left or right is used)

In [ ]:
main_path = "femaleAdventurer_walk*.png"
glob.glob(main_path)

### (IV) Change the sprites, sounds and then modify the placement of these items to make your own level

In [ ]:
#use this area if you need to check some code before you enter it into the PlayerCharacter class